# Test to see how the IFSharp works 
Here I'll try to see if it's possible to substitute the fsharp files with IFSharp

First reference libraries and open namespace

In [2]:
// test on forward ray tacing
#r @"C:\Users\Jose M. Gonzalez\OneDrive\Phd\render\ray casting\Sample parts for version 2\Library1\Types\bin\Debug\Types.dll"
#r @"C:\Users\Jose M. Gonzalez\OneDrive\Phd\render\ray casting\Sample parts for version 2\Library1\RayTracing\bin\Debug\RayTracing.dll"
// libraries referenced


open Types.Algebra
open Types.ObjectTypes
open Types.types
open RayTracing.intersections
open Microsoft.FSharp.Data.UnitSystems.SI.UnitSymbols
open ForwardRayTracing
open SaveSensorInfo
open Random
open System.IO

*Create a **ray** and **material library** that will be the base to create the new rays*

In [3]:
let r = {
         Wavelenght = WaveLength(5e-7<m>);
         from = Point(0.,00.00,0.0); uvec = UnitVector(1.,1.,0.);
         MaxLength = infi;
         OpticalPathTravelled = 0.<m>;
         NumBounces = 0uy; bounces = [];
         MaxDispersions = 1uy;
         NumOfParticles = 0;
         IndexOfRefraction = 1.
         NoiseAdd= [||]
        }

let mout = [|{MatName= "Mirror"; R=1.; T=0.; n=(2.,WaveLength(5e-6 |> LanguagePrimitives.FloatWithMeasure<m>));LambPPM= 0.};
             {MatName= "Glass"; R=0.; T=1.; n=(1.3,WaveLength(5e-6 |> LanguagePrimitives.FloatWithMeasure<m>));LambPPM= 0.};
             {MatName= "air"; R=0.; T=1.; n=(1.,WaveLength(5e-6 |> LanguagePrimitives.FloatWithMeasure<m>));LambPPM= 0.}
             |]

let mat = dict (mout|> Array.map(fun x -> (x.MatName,x)))

In [4]:
mat.["air"]

{T = 1.0;
 R = 0.0;
 n = (1.0, WaveLength 5e-06);
 LambPPM = 0.0;
 MatName = "air";}

- ## Simulate a simple reflection on a flat surface

How? Some rays are generated following a gaussian distribution and in a direction that after a reflection with the first flat surface, they will hit on the circular sensor  
```
 tes

```

Define the objects

In [5]:
//
//
// Next test the rays are reflected on a flat surface
//
//

let nNRM = UnitVector(0.,0.,1.)
let Pground = Point(0.,0.,0.)
let ground = disc(Pground,100.,nNRM,"Mirror",Sensor(),[|(0uy, 0.0)|])  // Disc that represents the ground

// Source of Light
//  &
// sensor

let spoint = Point(3.,0.,3.)        // Sensor point centre
let lpoint = Point(-3.,0.,3.)       // Light source centre
let snrm = UnitVector(-0.5,0.,-1.)   // Sensor Direction
let lnrm = UnitVector(1.,0.,-1.)    // Light Direction
let sdisc = disc(spoint,3.5,snrm,true)      // Sensor
lnrm
let objs3 = [|Disc(ground);Disc(sdisc)|]


Define the rays we will use and perform the intersections

In [6]:

let rotLight = Matrix.RotateVector(UnitVector(0.,0.,1.),lnrm)
let manyRay3 = [|0..100000|]   // generate some rays on different directions from random points on the initial surface
               |> Array.map(fun x -> let pos = 
                                            let ipos = Samp2DGauss(0.1,0.)
                                            let ip= Point(ipos.[0],ipos.[1],0.)
                                            //let ip= Point(0.2,0.2,0.)
                                            rotLight.RotatePoint(ip).MoveAndCreateNew(lpoint)
                                     ({r with uvec=lnrm; from = pos}))

manyRay3.[0].uvec
manyRay3|> Array.iter( fun x -> ForwardRay(x,objs3,mat))


<null>

### Process and Plot the result

In [9]:
let data3 =
    match (objs3.[1]) with
    | Disc x -> let vox = {Pmin = Point(-x.Radius,-x.Radius,0.); Pmax = Point(x.Radius,x.Radius,0.)}
                Some(x.Sensor.SavedData,vox,x.Normal, x.Centre, x.Radius , 450, 450)
    | _ -> None


//let dat, _, uv2,pt2,rd2,_ ,_= match data3 with Some x -> x

Since I have used the Some(x)/None to avoid the error on the output of the match (because I ignore the other object types), I am  accesing to the content of Some(x) matching it on the next code:

In [12]:
let ndat = match data3 with Some x -> x

input.fsx(1,18): warning FS0025: Incomplete pattern matches on this expression. For example, the value 'None' may indicate a case not covered by the pattern(s).

In [13]:

let snsrs2 = CreateImage_Points_from_disk(ndat)
let pa = Path.Combine( __SOURCE_DIRECTORY__,"teeest.png")
SensorToImage(snsrs2,pa, 450,450)


<null>

Here I have found a problem with the forms on IFSharp. It looks like the forms are not well generated and they produce an empty form. Luckily the file output works and it's correctly saved.  
Since the image is saved, then the best option to solve visualize it, it's just using the Markdown syntaxis

![Previous image](teeest.png)